In [ ]:
import boto3

In [129]:
dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")

In [12]:
coffe_config_table = dynamodb.Table("serverlesspresso-config-table")

In [27]:
not coffe_config_table.get_item(
         Key={"PK": "config"},
        ).get("Item")["storeOpen"]

False

In [43]:
user_id = 5
event_id = "AWS"

In [56]:
events_client = boto3.client('events')
import json

In [60]:
cancel_order_payload =   [
    {
      "Detail": 
        json.dumps({"Message": "Shop is closed or not available",
        "userId.$": user_id,
        "eventId.$": event_id})
      ,
      "DetailType": "OrderProcessor.ShopUnavailable",
      "EventBusName": "Serverlesspresso",
      "Source": "awsserverlessda.serverlesspresso"
    }
  ]

In [61]:
events_client.put_events(Entries=cancel_order_payload)

{'FailedEntryCount': 0,
 'Entries': [{'EventId': '6f48b35f-4772-1965-35fd-6704b84c3752'}],
 'ResponseMetadata': {'RequestId': 'ba43f734-9410-456c-a497-151fdbe55347',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ba43f734-9410-456c-a497-151fdbe55347',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Thu, 24 Aug 2023 10:04:01 GMT'},
  'RetryAttempts': 0}}

In [62]:
cancel_order_payload

[{'Detail': '{"Message": "Shop is closed or not available", "userId.$": 5, "eventId.$": "AWS"}',
  'DetailType': 'OrderProcessor.ShopUnavailable',
  'EventBusName': 'Serverlesspresso',
  'Source': 'awsserverlessda.serverlesspresso'}]

In [74]:
events_payload =  {
      "EventBusName": "blub",
      "Source": "blab",
    }


In [75]:
    Message = "The shop was not ready, and so a 'not ready' event is emitted to cancel the current order."
    event_details = {
            "Details" : json.dumps(dict(Message = Message)),
            "DetailType": "OrderProcessor.ShopUnavailable",
        }
    Entries = events_payload

In [76]:
Entries

{'EventBusName': 'blub', 'Source': 'blab'}

In [77]:
Entries = events_payload | event_details

In [78]:
Entries

{'EventBusName': 'blub',
 'Source': 'blab',
 'Details': '{"Message": "The shop was not ready, and so a \'not ready\' event is emitted to cancel the current order."}',
 'DetailType': 'OrderProcessor.ShopUnavailable'}

In [79]:
from pprint import pprint

In [81]:
pprint("""\n#set($state = $input.params('state'))\n#set($eventId\
            \ = $input.params('eventId'))\n\n\n#set ($eventAndState = \"${eventId}-$state\"\
            )\n\n#if( $input.params('maxItems').toString() != \"\" )\n  #set($maxItems\
            \ = $input.params('maxItems'))\n#else\n  #set($maxItems = 100)\n#end\n\
            \n\n{\n    \"TableName\": \"serverlesspresso-order-table\",\n    \"IndexName\"\
            : \"GSI-status\",\n    \"KeyConditionExpression\": \"#ORDERSTATE = :ORDERSTATE\"\
            ,\n    \"ExpressionAttributeNames\": {\n    \"#ORDERSTATE\": \"ORDERSTATE\"\
            \n    },\n    \"ExpressionAttributeValues\": {\n    \":ORDERSTATE\": {\n\
            \      \"S\": \"$eventAndState\"\n    }\n    },\n    \"ScanIndexForward\"\
            : true,\n    \"Limit\": $maxItems,\n    \"ProjectionExpression\": \"PK,\
            \ SK, orderNumber, robot, drinkOrder, ORDERSTATE, TS, userId, baristaUserId\"\
            \n}""")

('\n'
 "#set($state = $input.params('state'))\n"
 "#set($eventId            \\ = $input.params('eventId'))\n"
 '\n'
 '\n'
 '#set ($eventAndState = "${eventId}-$state"            )\n'
 '\n'
 '#if( $input.params(\'maxItems\').toString() != "" )\n'
 "  #set($maxItems            \\ = $input.params('maxItems'))\n"
 '#else\n'
 '  #set($maxItems = 100)\n'
 '#end\n'
 '            \n'
 '\n'
 '{\n'
 '    "TableName": "serverlesspresso-order-table",\n'
 '    "IndexName"            : "GSI-status",\n'
 '    "KeyConditionExpression": "#ORDERSTATE = :ORDERSTATE"            ,\n'
 '    "ExpressionAttributeNames": {\n'
 '    "#ORDERSTATE": "ORDERSTATE"            \n'
 '    },\n'
 '    "ExpressionAttributeValues": {\n'
 '    ":ORDERSTATE": {\n'
 '            \\      "S": "$eventAndState"\n'
 '    }\n'
 '    },\n'
 '    "ScanIndexForward"            : true,\n'
 '    "Limit": $maxItems,\n'
 '    "ProjectionExpression": "PK,            \\ SK, orderNumber, robot, '
 'drinkOrder, ORDERSTATE, TS, userId, baris

In [98]:
item = {
    "PK": {
      "M": "order-queue"
    },
    "IDvalue": {
      "N": "0"
    }
  }
import os
coffee_config = dict(
    config_table=os.environ.get(
        "SERVERLESSPRESSO_CONFIG_TABLE",
        "serverlesspresso-config-table",
    ),
    counting_table=os.environ.get(
        "SERVERLESSPRESSO_COUNTING_TABLE",
        "serverlesspresso-counting-table",
    ),
    order_table=os.environ.get(
        "SERVERLESSPRESSO_ORDER_TABLE",
        "serverlesspresso-order-table",
    ),
    validator_table=os.environ.get(
        "SERVERLESSPRESSO_VALIDATOR_TABLE", 
        "serverlesspresso-validator-table"
    ),
    eventbridge = os.environ.get(
        "SERVERLESSPRESSO_EVENTBRIDGE", 
        "Serverlesspresso"
    ),
    eventsource = os.environ.get(
        "SERVERLESSPRESSO_EVENTSOURCE", 
        "awsserverlessda.serverlesspresso"
    )
    
)

In [99]:
coffe_counting_table = dynamodb.Table(coffee_config["counting_table"])

In [111]:
coffe_counting_table.put_item(Item={'PK': 'order-queue','IDvalue':  coffe_counting_table.get_item(
         Key={"PK": "order-queue"},
        ).get("Item")["IDvalue"] +1})


{'ResponseMetadata': {'RequestId': '2388HQJR0FDKEAHF2H7R0PCJ53VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 24 Aug 2023 14:19:26 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2388HQJR0FDKEAHF2H7R0PCJ53VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [112]:
coffe_counting_table.get_item(
         Key={"PK": "order-queue"},
        ).get("Item")["IDvalue"] 

Decimal('1')

In [106]:
coffe_counting_table.update_item(
  Key={"PK": "order-queue"},
  ExpressionAttributeValues = { ":inc": {"IDvalue": "1"} },
  UpdateExpression= "ADD loginCount :inc"
)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: Invalid UpdateExpression: Incorrect operand type for operator or function; operator: ADD, operand type: MAP, typeSet: ALLOWED_FOR_ADD_OPERAND

In [109]:
coffe_counting_table.update_item(
    Key= {"PK": "order-queue"},
    UpdateExpression="SET order-queue = \"order-queue\" + :inc",

    ExpressionAttributeValues={
        ':inc': 1,
    },
)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: Invalid UpdateExpression: Syntax error; token: """, near: "SET "order"

In [119]:
response = coffe_counting_table.update_item(
    Key={
        'PK': {'S': "order-queue"}
    },
    UpdateExpression='SET IDvalue = IDvalue + :incr',
    ExpressionAttributeValues={
        ':incr': 1
    },
    ReturnValues='UPDATED_NEW'
)

ClientError: An error occurred (ValidationException) when calling the UpdateItem operation: The provided key element does not match the schema

In [124]:
if not 0:
    print("blub")

blub


In [132]:
import boto3
import json
import time


order_table = dynamodb.Table("serverlesspresso-order-table")


def lambda_handler():
    order_table.update_item(
        Key={"PK": "orders", "SK": "042deba4-71df-4847-8e1c-70ac0926dfec"},
        UpdateExpression="set ORDERSTATE = :state, TS = :TS, reason = :reason",
        ExpressionAttributeValues={
            ":state": "berlin" + "-CANCELLED",
            ":reason": "blub",
            ":TS": int(time.time()),
        },
        ReturnValues="ALL_NEW",
    )

    return {"success": True}

In [133]:
lambda_handler()

{'success': True}